In [ ]:
pip install SpeechRecognition pydub

In [ ]:
import speech_recognition as sr
import os
import subprocess
from google.colab import files
from pydub import AudioSegment
from pydub.silence import split_on_silence


In [ ]:
files.upload()

Saving hello.mp3 to hello (3).mp3


{'hello.mp3': b'\xff\xf3D\xc4\x00\x00\x00\x03H\x01@\x00\x00\xff0\x88\x9c\xc6bS\x17\x98\x8e\x9d\xc4\xfe\t\x04\xe5\xe6\xde+\x07 \xbf\xec\xc3\xe0\xc0\x10(\x11\xc2CO\x07\xc8K\xc6\x864p|\x98\xcf\xccc\xf1a\x87\x9ag\xef\xf1,\xab\x9eL\xe7_\x11O\x7f\xc2\x8d\x0f9\x9f\xfe5\x96Y\xff\xfd>\xb5\x07\x07\x00 Xp\x081\x04n\xff\xf3D\xc4S\x12!\x1eT\x01\x9c8\x000\xb3\x10\x0e\xd5L2\nK*`\x90\x10f\x8b6a\xb8\x8e\xb7\x9a\xe2G\x18R\x18\xa3\xad\xa8\xccm\x91\xca\xae\xf6\xec1"\xa4\x84\xc0u\xa8\xe6z\xf4\x99>\xca\x9a\xc2\x16\xdf\xea\x99>\xd93H\n6Y\xf0\x995\x19\\\xdf\x88\xa2\x84\xb0\xed\xbcY\xd7h\xeb#\xc7\xad\xd4\xf4\xff\xf3D\xc4] \xf1\xfe\\\x01\xddx\x00\xa4\xd8\xbd\xfb\xff\xbd\xe3_\xdf\xe3X\xc7W\xdf\x1f\xdf\xef\xda\xb4\xff\x18\xde>5\xff\xbf\xdf\xef\xf5\xbcGM\x8f_\xef\xff\xec\xff\xa9%\x06Ge\x8c\xa8\xc19C\xb6\x10\x04\x83\xaala\x80\x11\x89!f\xc3\x1b\x8d\r\xcb\xc0\xa5a`\xd8\x8d\x1b\xec\xca5!sv\xf1\xbe\xcd\x95\xa3\xff\xf3D\xc4,\x1b\xd1\xeet\x00\xe3\xda\x98\xf6S\xad\x0f\x91\xac6Ud\xb2\x0br@\x85\xb9\xfa\xbdT0M"\x99v\xcf\x9d7

In [ ]:
# convert mp3 to wav file
subprocess.call(['ffmpeg', '-i', 'hello.mp3','wav_file.wav'])

0

In [ ]:


# initialize the recognizer
r = sr.Recognizer()

# a function that splits the audio file into chunks
# and applies speech recognition

def get_large_audio_transcription(path):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    # open the audio file using pydub
    sound = AudioSegment.from_wav(path)
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text
    # return the text for all chunks detected
    return whole_text


path = "/content/wav_file.wav"
print("\nFull text:", get_large_audio_transcription(path))



audio-chunks/chunk1.wav : Machine learning is a branch of artificial intelligence. 
audio-chunks/chunk2.wav : Ai in computer science which. 
audio-chunks/chunk3.wav : Focus is on the use of data and algorithms to imitate the way that humans learn. 
audio-chunks/chunk4.wav : Gradually improving its accuracy. 

Full text: Machine learning is a branch of artificial intelligence. Ai in computer science which. Focus is on the use of data and algorithms to imitate the way that humans learn. Gradually improving its accuracy. 
